# Feature Engineering for ML Dataset

Transform raw features into model-ready inputs:
- **CNA**: Discretize to -1/0/1 (loss/neutral/gain)
- **Microbiome**: CLR transform + z-score alpha diversity
- **Clinical**: One-hot encoding + age scaling
- **Drug**: Morgan fingerprints from SMILES

In [45]:
# Setup
import os, json, re
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler, LabelEncoder

# RDKit for molecular fingerprints
from rdkit import Chem
from rdkit.Chem import AllChem, rdFingerprintGenerator

RAW = Path(r"C:\Users\aaron\cancerchemo\data\raw")
PROC = Path(r"C:\Users\aaron\cancerchemo\data\processed")
ART = Path(r"C:\Users\aaron\cancerchemo\artifacts")

for p in [PROC, ART]:
    p.mkdir(parents=True, exist_ok=True)

print("✓ Environment ready")

✓ Environment ready


## Load ML Dataset

In [46]:
# Load the complete ML dataset
ml_data = pd.read_csv(PROC / "ml_dataset_final_with_micro.csv")

print(f"Dataset shape: {ml_data.shape}")
print(f"\nColumns: {ml_data.columns.tolist()[:20]}...")  # Show first 20
print(f"\nTarget variable (effectiveness_score) distribution:")
print(ml_data['effectiveness_score'].describe())

ml_data.head(3)

Dataset shape: (4032, 1262)

Columns: ['Patient_ID', 'drug_raw', 'drug_start_int', 'treatment_line', 'drugs_in_regimen', 'is_combination', 'monoclonal', 'has_investigational', 'has_non_standard_formulation', 'has_other_drug', 'regimen_drugs', 'pfs_months', 'pfs_status', 'reg_row_idx', 'effectiveness_score', 'smiles', 'drug_1_name', 'drug_1_smiles', 'drug_2_name', 'drug_2_smiles']...

Target variable (effectiveness_score) distribution:
count    4032.000000
mean        0.008238
std         0.671285
min        -1.000000
25%        -1.000000
50%         0.167150
75%         0.497100
max         1.000000
Name: effectiveness_score, dtype: float64


C:\Users\aaron\AppData\Local\Temp\ipykernel_32084\3640870167.py:2: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  ml_data = pd.read_csv(PROC / "ml_dataset_final_with_micro.csv")


,Patient_ID,drug_raw,drug_start_int,treatment_line,drugs_in_regimen,is_combination,monoclonal,has_investigational,has_non_standard_formulation,has_other_drug,...,CNA_XRCC2,CNA_YAP1,CNA_YES1,CNA_ZFHX3,CNA_ZNRF3,CNA_ZRSR2,genus_Bacteroides,genus_Streptococcus,genus_Enterococcus,alpha_diversity
0,GENIE-DFCI-000013,Carboplatin,29427.0,1,2,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.766949,0.164865,0.068186,0.683803
1,GENIE-DFCI-000136,Carboplatin,25365.0,1,2,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.715017,0.045732,0.239251,0.723121
2,GENIE-DFCI-000142,Cisplatin,24366.0,1,2,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.322117,0.445950,0.231932,1.063958


## 1. Process CNA Features

Discretize copy number alterations to categorical values:
- `-1` = Loss (< -0.5 log2 ratio)
- `0` = Neutral (-0.5 to 0.5)
- `1` = Gain (> 0.5 log2 ratio)

In [47]:
# Identify CNA columns
cna_cols = [c for c in ml_data.columns if c.startswith('CNA_')]
print(f"Found {len(cna_cols)} CNA features")

# Discretize CNA values
ml_data_processed = ml_data.copy()

for col in cna_cols:
    # Discretize: -1 (loss), 0 (neutral), 1 (gain)
    ml_data_processed[col] = ml_data[col].apply(
        lambda x: -1 if x < -0.5 else (1 if x > 0.5 else 0)
    )

# Check distribution
print("\nCNA value distribution (first 5 genes):")
for col in cna_cols[:5]:
    counts = ml_data_processed[col].value_counts().sort_index()
    print(f"  {col}: Loss={counts.get(-1, 0)}, Neutral={counts.get(0, 0)}, Gain={counts.get(1, 0)}")

print("\n✓ CNA features discretized to -1/0/1")

Found 505 CNA features

CNA value distribution (first 5 genes):
  CNA_ABL1: Loss=134, Neutral=3863, Gain=35
  CNA_ACVR1: Loss=10, Neutral=3993, Gain=29
  CNA_AGO1: Loss=0, Neutral=4032, Gain=0
  CNA_AGO2: Loss=0, Neutral=4013, Gain=19
  CNA_AKT1: Loss=37, Neutral=3837, Gain=158

✓ CNA features discretized to -1/0/1

CNA value distribution (first 5 genes):
  CNA_ABL1: Loss=134, Neutral=3863, Gain=35
  CNA_ACVR1: Loss=10, Neutral=3993, Gain=29
  CNA_AGO1: Loss=0, Neutral=4032, Gain=0
  CNA_AGO2: Loss=0, Neutral=4013, Gain=19
  CNA_AKT1: Loss=37, Neutral=3837, Gain=158

✓ CNA features discretized to -1/0/1


## 2. Process Microbiome Features

Apply centered log-ratio (CLR) transformation to microbiome abundances and z-score normalize alpha diversity

In [48]:
# Identify microbiome columns
microbe_cols = [c for c in ml_data.columns if c.startswith('genus_')]
alpha_col = 'alpha_diversity'

print(f"Microbiome features: {microbe_cols}")
print(f"Alpha diversity column: {alpha_col}")

if microbe_cols and alpha_col in ml_data.columns:
    # CLR transformation for relative abundances
    # CLR(x) = log(x / geometric_mean(x))
    
    def clr_transform(row):
        """Centered log-ratio transformation"""
        values = row[microbe_cols].values
        # Add pseudocount to avoid log(0)
        values_pseudo = values + 1e-6
        # Geometric mean
        geom_mean = np.exp(np.mean(np.log(values_pseudo)))
        # CLR transform
        clr_values = np.log(values_pseudo / geom_mean)
        return pd.Series(clr_values, index=[f"{c}_clr" for c in microbe_cols])
    
    # Apply CLR transform
    clr_features = ml_data_processed[microbe_cols].apply(clr_transform, axis=1)
    
    # Add CLR features to dataset
    for col in clr_features.columns:
        ml_data_processed[col] = clr_features[col]
    
    # Z-score normalize alpha diversity
    ml_data_processed['alpha_diversity_zscore'] = zscore(
        ml_data_processed[alpha_col].fillna(ml_data_processed[alpha_col].mean())
    )
    
    print("\n✓ Microbiome features transformed:")
    print(f"  - CLR transformed: {len(microbe_cols)} features")
    print(f"  - Alpha diversity z-scored")
    print(f"\nCLR feature stats:")
    print(clr_features.describe())
else:
    print("⚠ No microbiome features found")

Microbiome features: ['genus_Bacteroides', 'genus_Streptococcus', 'genus_Enterococcus']
Alpha diversity column: alpha_diversity

✓ Microbiome features transformed:
  - CLR transformed: 3 features
  - Alpha diversity z-scored

CLR feature stats:
       genus_Bacteroides_clr  genus_Streptococcus_clr  genus_Enterococcus_clr
count            4032.000000              4032.000000             4032.000000
mean                0.677392                 0.186857               -0.864249
std                 0.711326                 0.770731                1.024276
min                -2.108608                -2.782815               -6.614228
25%                 0.219446                -0.290489               -1.328099
50%                 0.638639                 0.196335               -0.684254
75%                 1.108008                 0.688387               -0.207271
max                 3.346262                 3.369640                1.519806

✓ Microbiome features transformed:
  - CLR transform

C:\Users\aaron\AppData\Local\Temp\ipykernel_32084\1987864879.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ml_data_processed[col] = clr_features[col]
C:\Users\aaron\AppData\Local\Temp\ipykernel_32084\1987864879.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ml_data_processed[col] = clr_features[col]
C:\Users\aaron\AppData\Local\Temp\ipykernel_32084\1987864879.py:28: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

## 3. Process Clinical Features

Encode categorical variables and scale continuous features

In [49]:
# Clinical features - comprehensive list
categorical_features = [
    'sex', 'race', 'ethnicity', 
    'smoking_history',
    'stage_at_diagnosis', 'pre_treatment_stage',
    'histology_type', 'histology_detailed',
    'tumor_grade',
    'distant_mets',
    'tumor_laterality',  # Left/Right/Bilateral tumor location
    'treatment_center'   # Institution (DFCI/MSK/UHN/VICC)
]

continuous_features = ['age_at_diagnosis', 'n_cancers', 'n_cancers_index']

# Binary metastasis features (Yes/No -> 1/0)
binary_features = [
    'mets_brain', 'mets_liver', 'mets_lung', 'mets_bone', 'mets_adrenal',
    'mets_lymph', 'mets_pleura', 'mets_other', 'mets_subq_tissue'
]

# Find which features exist
existing_categorical = [f for f in categorical_features if f in ml_data_processed.columns]
existing_continuous = [f for f in continuous_features if f in ml_data_processed.columns]
existing_binary = [f for f in binary_features if f in ml_data_processed.columns]

print(f"Categorical features to encode: {len(existing_categorical)}")
print(f"  {existing_categorical}")
print(f"\nContinuous features to scale: {len(existing_continuous)}")
print(f"  {existing_continuous}")
print(f"\nBinary features to encode: {len(existing_binary)}")
print(f"  {existing_binary}")

# 1. One-hot encode categorical features
print("\n" + "="*60)
print("ENCODING CATEGORICAL FEATURES")
print("="*60)
for feat in existing_categorical:
    # Fill missing with 'Unknown'
    ml_data_processed[feat] = ml_data_processed[feat].fillna('Unknown').astype(str)
    
    # One-hot encode
    dummies = pd.get_dummies(ml_data_processed[feat], prefix=feat, drop_first=False)
    
    # Add to dataset
    for col in dummies.columns:
        ml_data_processed[col] = dummies[col].astype(np.int8)
    
    print(f"  ✓ {feat}: {len(dummies.columns)} categories")

# 2. Scale continuous features
print("\n" + "="*60)
print("SCALING CONTINUOUS FEATURES")
print("="*60)
for feat in existing_continuous:
    # Fill missing with median
    median_val = ml_data_processed[feat].median()
    scaler = StandardScaler()
    ml_data_processed[f"{feat}_scaled"] = scaler.fit_transform(
        ml_data_processed[feat].fillna(median_val).values.reshape(-1, 1)
    )
    print(f"  ✓ {feat}: scaled (mean={ml_data_processed[f'{feat}_scaled'].mean():.3f}, std={ml_data_processed[f'{feat}_scaled'].std():.3f})")

# 3. Binary encode metastasis features
print("\n" + "="*60)
print("ENCODING BINARY FEATURES (Yes/No -> 1/0)")
print("="*60)
for feat in existing_binary:
    # Convert Yes/No to 1/0, missing to 0
    ml_data_processed[f"{feat}_binary"] = ml_data_processed[feat].apply(
        lambda x: 1 if str(x).strip().lower() == 'yes' else 0
    ).astype(np.int8)
    
    n_positive = (ml_data_processed[f"{feat}_binary"] == 1).sum()
    print(f"  ✓ {feat}: {n_positive} positive cases")

print("\n" + "="*60)
print("✓ ALL CLINICAL FEATURES ENCODED AND SCALED")
print("="*60)

Categorical features to encode: 12
  ['sex', 'race', 'ethnicity', 'smoking_history', 'stage_at_diagnosis', 'pre_treatment_stage', 'histology_type', 'histology_detailed', 'tumor_grade', 'distant_mets', 'tumor_laterality', 'treatment_center']

Continuous features to scale: 3
  ['age_at_diagnosis', 'n_cancers', 'n_cancers_index']

Binary features to encode: 9
  ['mets_brain', 'mets_liver', 'mets_lung', 'mets_bone', 'mets_adrenal', 'mets_lymph', 'mets_pleura', 'mets_other', 'mets_subq_tissue']

ENCODING CATEGORICAL FEATURES
  ✓ sex: 2 categories
  ✓ race: 8 categories
  ✓ ethnicity: 5 categories
  ✓ smoking_history: 6 categories
  ✓ stage_at_diagnosis: 6 categories
  ✓ pre_treatment_stage: 4 categories
  ✓ histology_type: 6 categories
  ✓ histology_detailed: 33 categories
  ✓ tumor_grade: 5 categories
  ✓ distant_mets: 4 categories
  ✓ tumor_laterality: 6 categories
  ✓ treatment_center: 4 categories

SCALING CONTINUOUS FEATURES
  ✓ age_at_diagnosis: scaled (mean=0.000, std=1.000)
  ✓ n_ca

C:\Users\aaron\AppData\Local\Temp\ipykernel_32084\1940846546.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ml_data_processed[col] = dummies[col].astype(np.int8)
C:\Users\aaron\AppData\Local\Temp\ipykernel_32084\1940846546.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  ml_data_processed[col] = dummies[col].astype(np.int8)
C:\Users\aaron\AppData\Local\Temp\ipykernel_32084\1940846546.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

## 4. Process Drug Features - Morgan Fingerprints

Generate Morgan fingerprints from SMILES codes for each drug in the dataset

In [50]:
# Load SMILES cache
with open(PROC / "drug_smiles_cache.json", 'r') as f:
    smiles_cache = json.load(f)

print(f"Loaded SMILES for {len(smiles_cache)} drugs")

# Create Morgan fingerprint generator (new API)
morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

def smiles_to_morgan_fp(smiles, radius=2, nBits=2048):
    """Convert SMILES to Morgan fingerprint using MorganGenerator"""
    if smiles is None or pd.isna(smiles):
        return np.zeros(nBits, dtype=np.int8)
    
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return np.zeros(nBits, dtype=np.int8)
        
        # Generate Morgan fingerprint (ECFP) using new API
        fp = morgan_gen.GetFingerprint(mol)
        return np.array(fp, dtype=np.int8)
    except:
        return np.zeros(nBits, dtype=np.int8)

# Create drug fingerprint mapping
drug_fp_map = {}
drug_canonical_names = {}
no_smiles_drugs = []

print("\nGenerating Morgan fingerprints...")
for drug_name, smiles in smiles_cache.items():
    # Canonical name (clean up)
    canonical_name = drug_name.strip()
    drug_canonical_names[drug_name] = canonical_name
    
    # Generate fingerprint (even if SMILES is None - will return zeros)
    fp = smiles_to_morgan_fp(smiles, radius=2, nBits=2048)
    drug_fp_map[canonical_name] = fp
    
    # Track drugs with no SMILES
    if smiles is None or pd.isna(smiles) or np.sum(fp) == 0:
        no_smiles_drugs.append(canonical_name)
    
    if len(drug_fp_map) % 20 == 0:
        print(f"  Processed {len(drug_fp_map)} drugs...")

print(f"\n✓ Generated fingerprints for {len(drug_fp_map)} drugs")
print(f"⚠️  {len(no_smiles_drugs)} drugs have no SMILES (biologics, investigational, etc.)")
print(f"   Examples: {', '.join(no_smiles_drugs[:5])}")
print(f"  Fingerprint size: 2048 bits")

# Show example
example_drug = list(drug_fp_map.keys())[0]
example_fp = drug_fp_map[example_drug]
print(f"\nExample - {example_drug}:")
print(f"  Fingerprint: {example_fp[:20]}... (showing first 20 bits)")
print(f"  Bits set: {np.sum(example_fp)} / 2048")

Loaded SMILES for 81 drugs

Generating Morgan fingerprints...
  Processed 20 drugs...
  Processed 40 drugs...
  Processed 60 drugs...
  Processed 80 drugs...

✓ Generated fingerprints for 81 drugs
⚠️  0 drugs have no SMILES (biologics, investigational, etc.)
   Examples: 
  Fingerprint size: 2048 bits

Example - Abiraterone Acetate:
  Fingerprint: [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0]... (showing first 20 bits)
  Bits set: 54 / 2048


## 5. Add Drug Fingerprints to Dataset

For combination therapies, concatenate fingerprints for each drug

In [51]:
# Find drug SMILES columns
drug_smiles_cols = [c for c in ml_data_processed.columns if c.startswith('drug_') and c.endswith('_smiles')]
drug_name_cols = [c for c in ml_data_processed.columns if c.startswith('drug_') and c.endswith('_name')]

print(f"Drug SMILES columns: {drug_smiles_cols}")
print(f"Drug name columns: {drug_name_cols}")

# Generate fingerprints for each drug position
max_drugs = len(drug_smiles_cols)
print(f"\nMaximum drugs per regimen: {max_drugs}")

# For each treatment, create concatenated fingerprint
all_fps = []

for idx, row in ml_data_processed.iterrows():
    # Collect fingerprints for all drugs in this regimen
    regimen_fps = []
    
    for i in range(1, max_drugs + 1):
        smiles_col = f'drug_{i}_smiles'
        name_col = f'drug_{i}_name'
        
        if smiles_col in ml_data_processed.columns:
            drug_name = row.get(name_col)
            smiles = row.get(smiles_col)
            
            if pd.notna(drug_name) and drug_name in drug_fp_map:
                fp = drug_fp_map[drug_name]
            elif pd.notna(smiles):
                # Generate on-the-fly if not in map
                fp = smiles_to_morgan_fp(smiles)
            else:
                fp = np.zeros(2048, dtype=np.int8)
            
            regimen_fps.append(fp)
    
    # Concatenate all drug fingerprints
    if regimen_fps:
        combined_fp = np.concatenate(regimen_fps)
    else:
        combined_fp = np.zeros(2048 * max_drugs, dtype=np.int8)
    
    all_fps.append(combined_fp)
    
    if (idx + 1) % 500 == 0:
        print(f"  Processed {idx + 1} / {len(ml_data_processed)} treatments...")

# Convert to array
drug_fp_array = np.array(all_fps, dtype=np.int8)

print(f"\n✓ Drug fingerprint array shape: {drug_fp_array.shape}")
print(f"  (rows = treatments, cols = {2048} bits × {max_drugs} drugs)")

Drug SMILES columns: ['drug_1_smiles', 'drug_2_smiles', 'drug_3_smiles', 'drug_4_smiles']
Drug name columns: ['drug_1_name', 'drug_2_name', 'drug_3_name', 'drug_4_name']

Maximum drugs per regimen: 4
  Processed 500 / 4032 treatments...
  Processed 1000 / 4032 treatments...
  Processed 1500 / 4032 treatments...
  Processed 2000 / 4032 treatments...
  Processed 500 / 4032 treatments...
  Processed 1000 / 4032 treatments...
  Processed 1500 / 4032 treatments...
  Processed 2000 / 4032 treatments...
  Processed 2500 / 4032 treatments...
  Processed 3000 / 4032 treatments...
  Processed 3500 / 4032 treatments...
  Processed 4000 / 4032 treatments...

✓ Drug fingerprint array shape: (4032, 8192)
  (rows = treatments, cols = 2048 bits × 4 drugs)
  Processed 2500 / 4032 treatments...
  Processed 3000 / 4032 treatments...
  Processed 3500 / 4032 treatments...
  Processed 4000 / 4032 treatments...

✓ Drug fingerprint array shape: (4032, 8192)
  (rows = treatments, cols = 2048 bits × 4 drugs)


## 6. Save Processed Features

In [52]:
# Save processed dataset (without fingerprints - too large for CSV)
ml_data_processed.to_csv(PROC / "ml_dataset_processed.csv", index=False)
print(f"✓ Saved ml_dataset_processed.csv: {ml_data_processed.shape}")

# Save drug fingerprints as numpy array
np.save(ART / "drug_fingerprints.npy", drug_fp_array)
print(f"✓ Saved drug_fingerprints.npy: {drug_fp_array.shape}")

# Save drug mapping (name -> canonical name)
with open(ART / "drug_map.json", 'w') as f:
    json.dump(drug_canonical_names, f, indent=2)
print(f"✓ Saved drug_map.json: {len(drug_canonical_names)} drugs")

# Save individual drug fingerprints for lookup
drug_fp_dict = {name: fp.tolist() for name, fp in drug_fp_map.items()}
np.savez_compressed(
    ART / "drug_fp_library.npz",
    **{name: fp for name, fp in drug_fp_map.items()}
)
print(f"✓ Saved drug_fp_library.npz: {len(drug_fp_map)} drugs × 2048 bits")

print("\n" + "="*60)
print("FEATURE ENGINEERING COMPLETE")
print("="*60)

✓ Saved ml_dataset_processed.csv: (4032, 1367)
✓ Saved drug_fingerprints.npy: (4032, 8192)
✓ Saved drug_map.json: 81 drugs
✓ Saved drug_fp_library.npz: 81 drugs × 2048 bits

FEATURE ENGINEERING COMPLETE


## 7. Feature Summary

In [53]:
# Summarize all feature types
print("FEATURE SUMMARY")
print("="*60)

# Mutations (binary)
mut_cols = [c for c in ml_data_processed.columns if c.startswith('MUT_')]
print(f"\n1. MUTATIONS (binary):")
print(f"   {len(mut_cols)} features")

# CNA (discretized)
cna_cols = [c for c in ml_data_processed.columns if c.startswith('CNA_')]
print(f"\n2. COPY NUMBER ALTERATIONS (discretized -1/0/1):")
print(f"   {len(cna_cols)} features")

# Microbiome (CLR + z-score)
clr_cols = [c for c in ml_data_processed.columns if '_clr' in c]
alpha_zscore = 'alpha_diversity_zscore' in ml_data_processed.columns
print(f"\n3. MICROBIOME:")
print(f"   {len(clr_cols)} CLR-transformed abundances")
print(f"   {1 if alpha_zscore else 0} z-scored alpha diversity")

# Clinical (one-hot + scaled + binary)
clinical_onehot = [c for c in ml_data_processed.columns 
                   if any(c.startswith(f"{cat}_") for cat in existing_categorical)]
clinical_scaled = [c for c in ml_data_processed.columns if c.endswith('_scaled')]
clinical_binary = [c for c in ml_data_processed.columns if c.endswith('_binary')]
print(f"\n4. CLINICAL:")
print(f"   {len(clinical_onehot)} one-hot encoded categorical features")
print(f"   {len(clinical_scaled)} scaled continuous features")
print(f"   {len(clinical_binary)} binary metastasis features")

# Treatment metadata
treatment_cols = ['treatment_line', 'drugs_in_regimen', 'is_combination', 'monoclonal', 'pfs_months']
existing_treatment = [c for c in treatment_cols if c in ml_data_processed.columns]
print(f"\n5. TREATMENT METADATA:")
print(f"   {len(existing_treatment)} features: {existing_treatment}")

# Drug fingerprints
print(f"\n6. DRUG FINGERPRINTS (Morgan, radius=2):")
print(f"   Shape: {drug_fp_array.shape}")
print(f"   (2048 bits per drug × up to {max_drugs} drugs)")

# Target
print(f"\n7. TARGET VARIABLE:")
print(f"   effectiveness_score (continuous 0-1)")
print(f"   Mean: {ml_data_processed['effectiveness_score'].mean():.3f}")
print(f"   Std: {ml_data_processed['effectiveness_score'].std():.3f}")

# Total
total_features = len(mut_cols) + len(cna_cols) + len(clr_cols) + int(alpha_zscore) + \
                 len(clinical_onehot) + len(clinical_scaled) + len(clinical_binary) + \
                 len(existing_treatment) + drug_fp_array.shape[1]

print(f"\n" + "="*60)
print(f"TOTAL FEATURES: {total_features:,}")
print(f"TOTAL SAMPLES: {len(ml_data_processed):,}")
print("="*60)

FEATURE SUMMARY

1. MUTATIONS (binary):
   696 features

2. COPY NUMBER ALTERATIONS (discretized -1/0/1):
   505 features

3. MICROBIOME:
   3 CLR-transformed abundances
   1 z-scored alpha diversity

4. CLINICAL:
   89 one-hot encoded categorical features
   3 scaled continuous features
   9 binary metastasis features

5. TREATMENT METADATA:
   5 features: ['treatment_line', 'drugs_in_regimen', 'is_combination', 'monoclonal', 'pfs_months']

6. DRUG FINGERPRINTS (Morgan, radius=2):
   Shape: (4032, 8192)
   (2048 bits per drug × up to 4 drugs)

7. TARGET VARIABLE:
   effectiveness_score (continuous 0-1)
   Mean: 0.008
   Std: 0.671

TOTAL FEATURES: 9,503
TOTAL SAMPLES: 4,032


## 8. Create Final Feature Matrix for ML

Combine all features into a single matrix ready for training

In [54]:
# Select feature columns (exclude metadata and raw features)
exclude_patterns = [
    # Identifiers and metadata
    'Patient_ID', 'drug_raw', 'drug_start_int', 'reg_row_idx', 'birth_year',
    'regimen_drugs', 'drug_.*_name', 'drug_.*_smiles', 'smiles',
    
    # Raw categorical features (keep encoded versions with prefixes)
    '^sex$', '^race$', '^ethnicity$', '^smoking_history$', 
    '^stage_at_diagnosis$', '^pre_treatment_stage$',
    '^histology_type$', '^histology_detailed$', '^tumor_grade$', '^distant_mets$',
    '^tumor_laterality$', '^treatment_center$',
    
    # Raw continuous features (keep _scaled versions)
    '^age_at_diagnosis$', '^n_cancers$', '^n_cancers_index$',
    
    # Raw binary metastasis features (keep _binary versions)
    '^mets_brain$', '^mets_liver$', '^mets_lung$', '^mets_bone$', '^mets_adrenal$',
    '^mets_lymph$', '^mets_pleura$', '^mets_other$', '^mets_subq_tissue$',
    
    # Raw microbiome (keep CLR versions)
    '^genus_',  
    '^alpha_diversity$',  # Keep z-scored version
    
    # Target variables
    'pfs_status', 'pfs_months', 'effectiveness_score'
]

# Build feature columns
feature_cols = []
for col in ml_data_processed.columns:
    # Skip if matches any exclude pattern
    skip = False
    for pattern in exclude_patterns:
        if re.search(pattern, col):
            skip = True
            break
    
    if not skip:
        feature_cols.append(col)

print(f"Selected {len(feature_cols)} tabular features")
print(f"\nFeature categories included:")
print(f"  - Mutations: {len([c for c in feature_cols if c.startswith('MUT_')])}")
print(f"  - CNA: {len([c for c in feature_cols if c.startswith('CNA_')])}")
print(f"  - Microbiome CLR: {len([c for c in feature_cols if '_clr' in c])}")
print(f"  - Alpha diversity z-score: {len([c for c in feature_cols if c == 'alpha_diversity_zscore'])}")
print(f"  - Clinical one-hot: {len([c for c in feature_cols if any(c.startswith(f'{cat}_') for cat in existing_categorical)])}")
print(f"  - Clinical scaled: {len([c for c in feature_cols if c.endswith('_scaled')])}")
print(f"  - Clinical binary (mets): {len([c for c in feature_cols if c.endswith('_binary')])}")
print(f"  - Treatment metadata: {len([c for c in feature_cols if c in existing_treatment])}")

# Create feature matrix (tabular features only)
X_tabular = ml_data_processed[feature_cols].values

# Drug fingerprints (already in array)
X_drug_fp = drug_fp_array

# Target variable
y = ml_data_processed['effectiveness_score'].values

# Metadata (for tracking)
metadata_cols = ['Patient_ID', 'drug_raw', 'treatment_line', 'is_combination', 'monoclonal']
metadata = ml_data_processed[[c for c in metadata_cols if c in ml_data_processed.columns]]

print(f"\n✓ Feature matrices created:")
print(f"  X_tabular: {X_tabular.shape} (genomics + clinical + microbiome)")
print(f"  X_drug_fp: {X_drug_fp.shape} (drug fingerprints)")
print(f"  y: {y.shape} (effectiveness scores)")
print(f"  metadata: {metadata.shape}")

Selected 1318 tabular features

Feature categories included:
  - Mutations: 696
  - CNA: 505
  - Microbiome CLR: 0
  - Alpha diversity z-score: 1
  - Clinical one-hot: 89
  - Clinical scaled: 3
  - Clinical binary (mets): 9
  - Treatment metadata: 4

✓ Feature matrices created:
  X_tabular: (4032, 1318) (genomics + clinical + microbiome)
  X_drug_fp: (4032, 8192) (drug fingerprints)
  y: (4032,) (effectiveness scores)
  metadata: (4032, 5)

✓ Feature matrices created:
  X_tabular: (4032, 1318) (genomics + clinical + microbiome)
  X_drug_fp: (4032, 8192) (drug fingerprints)
  y: (4032,) (effectiveness scores)
  metadata: (4032, 5)


## 9. Save Final Feature Matrices

In [55]:
# Save feature matrices
np.save(ART / "X_tabular.npy", X_tabular.astype(np.float32))
np.save(ART / "X_drug_fp.npy", X_drug_fp)
np.save(ART / "y_effectiveness.npy", y.astype(np.float32))

# Save feature names
with open(ART / "feature_names.json", 'w') as f:
    json.dump({
        'tabular_features': feature_cols,
        'drug_fp_size': X_drug_fp.shape[1],
        'target': 'effectiveness_score'
    }, f, indent=2)

# Save metadata
metadata.to_csv(ART / "sample_metadata.csv", index=False)

print("✓ Saved feature matrices to artifacts/")
print(f"  - X_tabular.npy: {X_tabular.nbytes / 1024 / 1024:.1f} MB")
print(f"  - X_drug_fp.npy: {X_drug_fp.nbytes / 1024 / 1024:.1f} MB")
print(f"  - y_effectiveness.npy: {y.nbytes / 1024:.1f} KB")
print(f"  - feature_names.json")
print(f"  - sample_metadata.csv")

print("\n" + "="*60)
print("✓ FEATURE ENGINEERING PIPELINE COMPLETE")
print("="*60)
print("\nReady for model training!")
print("\nNext steps:")
print("  1. Split data into train/val/test sets")
print("  2. Train ML models (XGBoost, Random Forest, Neural Networks)")
print("  3. Evaluate model performance")
print("  4. Interpret feature importance")

✓ Saved feature matrices to artifacts/
  - X_tabular.npy: 40.5 MB
  - X_drug_fp.npy: 31.5 MB
  - y_effectiveness.npy: 31.5 KB
  - feature_names.json
  - sample_metadata.csv

✓ FEATURE ENGINEERING PIPELINE COMPLETE

Ready for model training!

Next steps:
  1. Split data into train/val/test sets
  2. Train ML models (XGBoost, Random Forest, Neural Networks)
  3. Evaluate model performance
  4. Interpret feature importance
